In [1]:
import pandas as pd
import os
import yaml

Need to set this up to overwrite whats in /projects/pangen/analysis/hmac/applications/Griffin/snakemakes/griffin_nucleosome_profiling/config

Overright:
- sites.yaml
- results.yaml 

Then run snakemake and have it output in your favourite directory 

In [2]:
def make_gene_input_tsvs(gene_list, output_dir, outfile):
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs("logs", exist_ok=True)
    
    gtf = pd.read_csv('/projects/pangen/analysis/hmac/reference/hg38_ref/genome.ucsc.regions.txt', sep = '\t')
    df = gtf[gtf['name2'].isin(gene_list)]
    df = df[df['chrom'].isin(chroms)]
    
    df = df[['chrom', 'txStart', 'name2', 'strand']]
    df.rename(columns={'chrom': 'Chrom', 'txStart': 'Start', 'name2': 'Gene', 'strand': 'Strand'}, inplace=True)
    df['End'] = df['Start'] + 1
    df['position'] = df['Start']
    df['uniprotId'] = 'unknown'

    # Reorder the DataFrame
    column_order = ["Chrom", "Start", "End", "uniprotId", "Gene", "position", 'Strand']
    df = df[column_order]
    df = df.drop_duplicates()

    # Count occurrences of each gene
    gene_counts = df["Gene"].value_counts()

    # Dictionary to track occurrences
    gene_counter = {}

    # Function to rename genes
    def rename_gene(gene):
        if gene_counts[gene] > 1:  # Only rename if there are duplicates
            if gene in gene_counter:
                gene_counter[gene] += 1
            else:
                gene_counter[gene] = 1
            return f"{gene}_{gene_counter[gene]}"  # Append _1, _2, etc.
        else:
            return gene  # Keep unique genes unchanged

    # Apply function to rename duplicates
    df["Gene"] = df["Gene"].apply(rename_gene)

    # Iterate over each row and save as a separate TSV file
    for index, row in df.iterrows():
        # Convert row to a single-row DataFrame
        row_df = pd.DataFrame([row])
        # Define the filename using the Gene name
        filename = os.path.join(output_dir, f"{row['Gene']}.tsv")
        # Save as TSV (tab-separated)
        row_df.to_csv(filename, sep='\t', index=False)

    # Save them all 
    df.to_csv(f"{output_dir}/{outfile}", sep="\t", index=False)
    return df

In [ ]:
name = 'housekeeping'
gene_list = ['ACTB', 'GAPDH', 'RPLP0']
tsvdir = 'site_files'
outfile = 'test.tsv'

chroms = [f"chr{i}" for i in range(1, 23)] + ["chrX", "chrY"]


In [ ]:
# Make tsvs for griff
tsvdir = 'site_files'
outfile = 'test.tsv'
df = make_gene_input_tsvs(gene_list, tsvdir, outfile)

In [11]:
from pathlib import Path
import os


output_file = Path("/projects/pangen/analysis/hmac/applications/Griffin/snakemakes/griffin_nucleosome_profiling/config/sites.yaml")
tsvdir = Path(".") / "site_files"

with output_file.open("w") as file:
    file.write("site_lists:\n")

    def write_paths(indir):
        for site in os.listdir(indir):
            name = site.split('.')[0]
            full_path = (indir / f"{name}.tsv").resolve()
            file.write(f"    {name}: {full_path}\n")

    write_paths(tsvdir)


In [ ]:
# MAke sites.yaml
output_file = "/projects/pangen/analysis/hmac/applications/Griffin/snakemakes/griffin_nucleosome_profiling/config/sites.yaml"

tsvdir = "site_files"

with open(output_file, "w") as file:
    file.write("site_lists:\n")  
    def write_paths(indir):
        for site in os.listdir(indir):
            name = site.split('.')[0]
            file.write(f"    {name}: {os.path.join(indir, name)}.tsv\n")
    write_paths(tsvdir)



FileNotFoundError: [Errno 2] No such file or directory: 'tsvdir'

In [12]:
import os
import yaml
from yaml.representer import SafeRepresenter

# Trick PyYAML into writing a plain scalar string
class PlainString(str):
    pass

def plain_string_representer(dumper, data):
    return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='')

yaml.add_representer(PlainString, plain_string_representer)

def write_griffin_config(output_dir, results_dir):
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(results_dir, exist_ok=True)

    chroms_str = PlainString(" ".join([f"chr{i}" for i in range(1, 23)] + ["chrX", "chrY"]))

    config = {
        'results_dir': results_dir,
    }

    with open(os.path.join(output_dir, 'results.yaml'), 'w') as f:
        yaml.dump(config, f, default_flow_style=False)

# Example usage
write_griffin_config('/projects/pangen/analysis/hmac/applications/Griffin/snakemakes/griffin_nucleosome_profiling/config', '/projects/pangen/analysis/hmac/applications/Griffin/test/results')


In [ ]:
from pathlib import Path
import os
import yaml

# Trick PyYAML into writing a plain scalar string
class PlainString(str):
    pass

def plain_string_representer(dumper, data):
    return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='')

yaml.add_representer(PlainString, plain_string_representer)

def write_griffin_config(output_dir):
    output_dir = Path(output_dir).resolve()
    results_dir = output_dir / "results"

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(results_dir, exist_ok=True)

    config = {
        'results_dir': str(results_dir),
    }

    with open(output_dir / 'results.yaml', 'w') as f:
        yaml.dump(config, f, default_flow_style=False)


In [ ]:
# Dont need i think

import yaml
from pathlib import Path

# Order matters: later keys override earlier ones if duplicated
config_files = [
    "/projects/pangen/analysis/hmac/applications/Griffin/snakemakes/griffin_GC_and_mappability_correction/results/samples.GC.yaml",
    "config/sites.yaml",
    "config/config.yaml",
    "/projects/pangen/analysis/hmac/applications/Griffin/snakemakes/griffin_nucleosome_profiling/config/cluster_slurm.yaml"
]

merged = {}
for path in config_files:
    with open(path, 'r') as f:
        content = yaml.safe_load(f)
        if content:
            merged.update(content)

# Output to a single combined config
with open("config/combined.yaml", "w") as out:
    yaml.dump(merged, out, default_flow_style=False)



# Run in your working directory - where all output will be made
```
snakemake -s /projects/pangen/analysis/hmac/Griffin/snakemakes/griffin_nucleosome_profiling/griffin_nucleosome_profiling.snakefile \
--jobs 40 \
  --configfile /projects/pangen/analysis/hmac/Griffin/snakemakes/griffin_GC_and_mappability_correction/results/samples.GC.yaml \
  --configfile sites.yaml \
  --configfile config.yaml \
  --configfile /projects/pangen/analysis/hmac/Griffin/snakemakes/griffin_nucleosome_profiling/config/cluster_slurm.yaml \
  --latency-wait 60 \
  --keep-going \
  --rerun-incomplete \
  --cluster-config /projects/pangen/analysis/hmac/Griffin/snakemakes/griffin_nucleosome_profiling/config/cluster_slurm.yaml \
  --cluster "sbatch --mem={cluster.mem} -t {cluster.time} -c {cluster.ncpus} -n {cluster.ntasks} -o {cluster.output} -J {cluster.JobName}"
```